In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# prompt Temlate
t = PromptTemplate.from_template("Tell me about the species about {animal}")
t.format(animal="hamster")


'Tell me about the species about hamster'

In [2]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
    },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAi:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Germany")


'Human: What do you know about France?\nAi:\nHere is what I know:\nCapital: Paris\nLanguage: French\nFood: Wine and Cheese\nCurrency: Euro\n\n\nHuman: What do you know about Italy?\nAi:\nI know this:\nCapital: Rome\nLanguage: Italian\nFood: Pizza and Pasta\nCurrency: Euro\n\n\nHuman: What do you know about Greece?\nAi:\nI know this:\nCapital: Athens\nLanguage: Greek\nFood: Souvlaki and Feta Cheese\nCurrency: Euro\n\n\nHuman: What do you know about Germany?'

In [3]:
chain = prompt | chat

chain.invoke({"country": "German"})


Ai:
I know this:
Capital: Berlin
Language: German
Food: Bratwurst and Sauerkraut
Currency: Euro

AIMessageChunk(content='Ai:\nI know this:\nCapital: Berlin\nLanguage: German\nFood: Bratwurst and Sauerkraut\nCurrency: Euro')

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Thailand"})


# 4.3 LengthBasedExampleSelector


In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

# 챗모델 설정 - 창의력/응답생성관리/등
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# 퓨샷 예시들
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

# 선제공예시 개수 조절 example_selector
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=80,
)
# 프롬프트
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)
# 프롬프트의 인풋변수 할당
prompt.format(country="Brazil")


'Human: What do you know about France?\nAI:\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

## 예시 랜덤 선택


In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector  # !

# 챗모델 설정 - 창의력/응답생성관리/등
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# 퓨샷 예시들
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# BaseExampleSelector - 사용자 맞춤 구현하기


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):  # 이미 존재하는 example에 example을 추가하는 함수인 add_example 필요
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice  # 무작위 예시 랜덤 선택

        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

# 선제공예시 개수 조절 example_selector
example_selector = RandomExampleSelector(
    examples=examples,
)
# 프롬프트
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)
# 프롬프트의 인풋변수 할당
prompt.format(country="Brazil")


'Human: What do you know about France?\nAI:\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

# 4.4 Serialization and Composition


In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.json")  # json

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

prompt.format(country="Germany")


'What is the capital of Germany'

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.yaml")  # yaml

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

prompt.format(country="Germany")


'What is the capital of Germany'

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=False,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(  # 작은 부분 프롬프트 1
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(  # 작은 부분 프롬프트 2
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(  # 작은 부분 프롬프트 3
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(  # 모든 프롬프트를 합친 최종 프롬프트
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)


AIMessage(content="Arrr matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothing beats the taste o' the sea! Arrrr!")

# 4.5 Caching


In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())


chat = ChatOpenAI(
    temperature=0.1,
)

chat.predict("How do you make italian pasta")  # 첫 번째 시도 4초


'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or toppings.\n\n

In [17]:
chat.predict("How do you make italian pasta")  # 두 번째 시도 즉시


'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or toppings.\n\n

### 데이터베이스를 사용하여 저장


In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)  # 디버그 로그

chat = ChatOpenAI(
    temperature=0.1,
)

chat.predict("How do you make italian pasta")


[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [4.00s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. 

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or toppings.\n\nEnjoy

# 4.6 Serialization


In [26]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)


with get_openai_callback() as usage:  # LLM을 콜백으로 불러옴
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, "\n")
    print(b, "\n")
    print(usage)  # 프롬프트가 얼마나 많은 토큰을 사용했는지 보여줌


Ingredients:
- 1 cup of rice
- 1 cup of water
- 1 tablespoon of nuruk (fermentation starter)
- 1 tablespoon of yeast

Instructions:
1. Rinse the rice thoroughly and soak it in water for at least 1 hour.
2. Drain the rice and steam it until it is fully cooked.
3. Let the rice cool down to room temperature.
4. In a large bowl, mix the nuruk and yeast with the water until well combined.
5. Add the cooked rice to the mixture and stir well.
6. Cover the bowl with a clean cloth and let it ferment in a warm place for 3-4 days.
7. After the fermentation process is complete, strain the mixture through a cheesecloth to remove any solids.
8. Transfer the liquid to a clean container and let it sit for another 1-2 days to allow the flavors to develop.
9. Your homemade soju is now ready to be enjoyed! Serve chilled and enjoy responsibly. 

Ingredients:
- 4 cups all-purpose flour
- 1 packet active dry yeast
- 1 1/2 cups warm water
- 2 tablespoons sugar
- 2 teaspoons salt
- 2 tablespoons olive oil

In